# Piano Technical Assignment - Python

In [1]:
##Import libraries
import requests
from requests.structures import CaseInsensitiveDict
import json
import pandas as pd

In [3]:
#Read FileA
fileA = pd.read_csv("FileA.csv")
fileA

,user_id,email
0,oi6IhEzu9R,user_2@example.com
1,fQFLNRDae8,user_5@example.com
2,fBYRtPtAlC,user_7@example.com
3,fOSjdLnNP3,user_9@example.com
4,uxz2jFwr5I,user_1@example.com
5,zSbmdNiSHH,user_4@example.com
6,fjM66woroy,user_0@example.com
7,oh4mHXh8zN,user_3@example.com
8,gXWj37JC5d,user_8@example.com
9,4dBdXURAz3,user_6@example.com


In [4]:
#Read FileB
fileB = pd.read_csv("FileB.csv")
fileB

,user_id,first_name,last_name
0,oh4mHXh8zN,Julie,Mosser
1,zSbmdNiSHH,Taryn,Jaycox
2,fBYRtPtAlC,John,Smith
3,fjM66woroy,Yadira,Irving
4,fQFLNRDae8,Vella,Lynam
5,fOSjdLnNP3,Qiana,Walk
6,uxz2jFwr5I,Benito,Festa
7,oi6IhEzu9R,Leatrice,Oquinn
8,4dBdXURAz3,Jacques,Cuellar
9,gXWj37JC5d,Shaun,Kreiger


In [10]:
#Merge the files
merged = fileA.join(fileB.set_index('user_id'), on='user_id')
merged

,user_id,email,first_name,last_name
0,oi6IhEzu9R,user_2@example.com,Leatrice,Oquinn
1,fQFLNRDae8,user_5@example.com,Vella,Lynam
2,fBYRtPtAlC,user_7@example.com,John,Smith
3,fOSjdLnNP3,user_9@example.com,Qiana,Walk
4,uxz2jFwr5I,user_1@example.com,Benito,Festa
5,zSbmdNiSHH,user_4@example.com,Taryn,Jaycox
6,fjM66woroy,user_0@example.com,Yadira,Irving
7,oh4mHXh8zN,user_3@example.com,Julie,Mosser
8,gXWj37JC5d,user_8@example.com,Shaun,Kreiger
9,4dBdXURAz3,user_6@example.com,Jacques,Cuellar


In [13]:
#Call Piano API to get correct userIDs
url = "https://sandbox.piano.io/api/v3/publisher/user/list"

headers = CaseInsensitiveDict()
headers["Content-Type"] = "application/x-www-form-urlencoded"
headers["Accept"] = "application/json"

data = "aid=o1sRRZSLlw&api_token=zziNT81wShznajW2BD5eLA4VCkmNJ88Guye7Sw4D&limit=100&offset=0"


resp = requests.post(url, headers=headers, data=data)
resp_dict = json.loads(resp.text)
print(resp.status_code)
resp_dict['users']

200


{'code': 0,
 'ts': 1623365785,
 'limit': 100,
 'offset': 0,
 'total': 7,
 'count': 9,
 'users': [{'first_name': None,
   'last_name': None,
   'personal_name': None,
   'email': 'has@access.com',
   'uid': 'PNIAHeDI8pjduak',
   'image1': None,
   'create_date': 1544212892,
   'reset_password_email_sent': False,
   'custom_fields': []},
  {'first_name': None,
   'last_name': None,
   'personal_name': None,
   'email': 'no@access.com',
   'uid': 'PNITyDyAOpjdubb',
   'image1': None,
   'create_date': 1544212919,
   'reset_password_email_sent': False,
   'custom_fields': []},
  {'first_name': None,
   'last_name': None,
   'personal_name': None,
   'email': 'user_0@example.com',
   'uid': 'PNISGXVjCpnhexk',
   'image1': None,
   'create_date': 1551098072,
   'reset_password_email_sent': True,
   'custom_fields': []},
  {'first_name': None,
   'last_name': None,
   'personal_name': None,
   'email': 'user_9@example.com',
   'uid': 'PNIABp6mSpnhexq',
   'image1': None,
   'create_date': 155

In [17]:
#Convert response to dataframe
apidata = pd.DataFrame.from_dict(resp_dict['users'])
apidata

,first_name,last_name,personal_name,email,uid,image1,create_date,reset_password_email_sent,custom_fields
0,None,None,None,has@access.com,PNIAHeDI8pjduak,None,1544212892,False,[]
1,None,None,None,no@access.com,PNITyDyAOpjdubb,None,1544212919,False,[]
2,None,None,None,user_0@example.com,PNISGXVjCpnhexk,None,1551098072,True,[]
3,None,None,None,user_9@example.com,PNIABp6mSpnhexq,None,1551098078,True,[]
4,None,None,None,user_5@example.com,PNIHNSuFZpnhey1,None,1551098089,True,[]
5,Matt,Hinton,Matt Hinton,matt.hinton@piano.io,PNIP7FSyVpoky83,None,1552942659,True,[]
6,allegra,anka,allegra anka,allegra.anka@piano.io,PNIYDhiQRqayeba,None,1590521591,False,[]
7,J,Mathews,J Mathews,test-login@assessments.io,PNIwGolcnqhazws,None,1601187149,True,[]
8,None,None,None,chessmastert@gmail.com,PNIMzXlM3qu4y4y,None,1622739923,True,[]


In [30]:
#Change incorrect userids with the API fetched userids
emllst = merged['email'].tolist()
for row in apidata.iterrows():
    if row['email'] in emllst:
        merged.loc[merged.email == row['email'], "user_id"] = row['uid']
merged

,user_id,email,first_name,last_name
0,oi6IhEzu9R,user_2@example.com,Leatrice,Oquinn
1,PNIHNSuFZpnhey1,user_5@example.com,Vella,Lynam
2,fBYRtPtAlC,user_7@example.com,John,Smith
3,PNIABp6mSpnhexq,user_9@example.com,Qiana,Walk
4,uxz2jFwr5I,user_1@example.com,Benito,Festa
5,zSbmdNiSHH,user_4@example.com,Taryn,Jaycox
6,PNISGXVjCpnhexk,user_0@example.com,Yadira,Irving
7,oh4mHXh8zN,user_3@example.com,Julie,Mosser
8,gXWj37JC5d,user_8@example.com,Shaun,Kreiger
9,4dBdXURAz3,user_6@example.com,Jacques,Cuellar


In [35]:
#Export file to csv
merged.to_csv('FileC.csv',index=False)